In [1]:
import os

os.getcwd()
os.chdir("../../")
os.getcwd()

'/data01/dl23vitcas/dl_project'

In [ ]:
import sys
import argparse
import os

sys.argv = ["view", "--config", "config/single_task_object_detection.yaml"]

parser = argparse.ArgumentParser()
parser.add_argument("--config", type=str, required=True, help="Path to the config file")
args = parser.parse_args()

print(args.config)

In [ ]:
from dataloader import VOC08Attr
from torchvision.transforms import transforms
from config_experiments import config
import torch
from torch.utils.data import DataLoader

import numpy as np

In [ ]:
import torch
import torchvision.transforms.functional as F
from torchvision.utils import draw_bounding_boxes
import matplotlib.pyplot as plt
import numpy as np


def show(imgs):
    plt.rcParams["savefig.bbox"] = "tight"
    if not isinstance(imgs, list):
        imgs = [imgs]
    fix, axs = plt.subplots(ncols=len(imgs), squeeze=False)
    for i, img in enumerate(imgs):
        img = img.detach()
        img = F.to_pil_image(img)  # (H, W)
        axs[0, i].imshow(np.asarray(img))  # (W, H, 3)
        # axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[]) # uncomment to remove axis in plot


def show_bbox_with_transform(image, box, mean, std, labels=None, color="white"):
    image = np.array(image)
    for channel in range(3):
        image[channel] = image[channel] * std[channel] + mean[channel]
    image = np.clip(image, 0, 1)
    image = (image * 255).astype(np.uint8)
    image = torch.from_numpy(image)
    show(draw_bounding_boxes(image, box, colors=color, labels=labels, width=2))

In [ ]:
from dataloader import VOC08Attr
import torchvision.transforms as transforms
from config_experiments import config


train_data = VOC08Attr(
    train=True,
    transform=transforms.Compose(
        [
            transforms.Resize(
                size=config["transform"]["resize_values"],
                max_size=config["transform"]["max_size"],
            ),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=config["transform"]["mean"], std=config["transform"]["std"]
            ),
        ]
    ),
)

In [ ]:
for i, (image, img_size_orig, gt_class, gt_bbox, gt_attributes, ss_rois) in enumerate(
    train_data
):
    show_bbox_with_transform(
        image, ss_rois, config["transform"]["mean"], std=config["transform"]["std"]
    )
    if i == 10:
        break